In [2]:
import sys
import argparse
import random
import copy
import pyro
import torch
import os
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, accuracy_score
import numpy as np
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'
sys.path.append('..')
sys.path.append('../..')
from causal_models.trainer import preprocess_batch
from train_setup import setup_directories, setup_tensorboard, setup_logging
from train_setup import setup_dataloaders
# From datasets import get_attr_max_min
from utils import EMA, seed_all
from dscm.dscm import DSCM
from hvae2 import HVAE2
import torch.nn.functional as F
from pgm.train_pgm import sup_epoch, eval_epoch
from pgm.utils_pgm import check_nan, update_stats, calculate_loss, plot_cf
from dscm.dscm import vae_preprocess
from pgm.layers import TraceStorage_ELBO
from pgm.chest_pgm_regressor import FlowPGM

from unet import ResUnet


### Setup dataloaders

In [3]:
class Hparams:
    def update(self, dict):
        for k, v in dict.items():
            setattr(self, k, v)

# Load deep VAE
beta = 3
vae_path = f"../checkpoints/Left-Lung_volume_Right-Lung_volume_Heart_volume/mimic_crop_256_64_beta_{beta}_segmentations/checkpoint.pt"

print(f'\nLoading VAE checkpoint: {vae_path}')
vae_checkpoint = torch.load(vae_path)
vae_args = Hparams()
vae_args.batch_size = 10

vae_args.update(vae_checkpoint['hparams'])



Loading VAE checkpoint: ../checkpoints/Left-Lung_volume_Right-Lung_volume_Heart_volume/mimic_crop_256_64_beta_3_segmentations/checkpoint.pt


In [4]:
dataloaders = setup_dataloaders(vae_args, cache=False, shuffle_train=False)

{'project_name': 'MIMIC_crop generation', 'seed': 11, 'mixed_precision': False, 'is_unit_test_config': False, 'data': {'batch_size': 16, 'num_workers': 12, 'pin_memory': True, 'input_channels': 1, 'weights': 'None', 'augmentations': {'resize': [256, 64], 'center_crop': 'None', 'random_rotation': 0, 'horizontal_flip': False, 'vertical_flip': False, 'random_crop': 'None', 'random_color_jitter': 0.1, 'random_erase_scale': [0.0, 0.0], 'sharp': 0.0}, 'prop_train': 1.0, '_target_': 'data_handling.chest_xray.MimicDataModule', 'dataset': 'mimic', 'domain': 'None', 'cache': False, 'seg_target_list': ['Left-Lung', 'Right-Lung', 'Heart']}, 'trainer': {'name': 'base', 'lr': 0.001, 'num_epochs': 400, 'patience_for_scheduler': 10, 'metric_to_monitor': 'Val/loss', 'metric_to_monitor_mode': 'min', 'val_check_interval': 'None', 'weight_decay': 0.0, 'use_train_augmentations': True, 'loss': 'ce', 'contrastive_temperature': 0.1, 'return_two_views': False, 'finetune_path': 'None', 'device': [0], 'max_steps

/vol/biomedic3/tx1215/heartflow_mimic_crop/causal_models/pgm/../../data_handling/chest_xray.py:129: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['disease'] = df['disease'].replace({'No Finding': 0, 'Pleural Effusion': 1})


sex
Male      0.516252
Female    0.483748
Name: proportion, dtype: float64
disease
No Finding          0.541031
Pleural Effusion    0.458969
Name: proportion, dtype: float64
race
White    0.764848
Black    0.188804
Asian    0.046348
Name: proportion, dtype: float64
Len dataset 9968
Test df: 


/vol/biomedic3/tx1215/heartflow_mimic_crop/causal_models/pgm/../../data_handling/chest_xray.py:129: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['disease'] = df['disease'].replace({'No Finding': 0, 'Pleural Effusion': 1})


sex
Male      0.522024
Female    0.477976
Name: proportion, dtype: float64
disease
No Finding          0.546651
Pleural Effusion    0.453349
Name: proportion, dtype: float64
race
White    0.772491
Black    0.186999
Asian    0.040511
Name: proportion, dtype: float64
Len dataset 30535


/vol/biomedic3/tx1215/heartflow_mimic_crop/causal_models/pgm/../../data_handling/chest_xray.py:129: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['disease'] = df['disease'].replace({'No Finding': 0, 'Pleural Effusion': 1})


In [5]:
for_count = 0
unet=ResUnet(channel=1)
for batch in tqdm(dataloaders["valid"]):
    for_count+=1
    seg = unet(batch['x'])
    if for_count>1:
        break


  0%|          | 1/615 [00:12<2:09:18, 12.64s/it]


In [35]:
print(seg.size())

torch.Size([16, 1, 256, 64])


In [55]:
sum_seg = torch.sum(seg, dim=(1,2,3), keepdim=True)

def get_min_max_valumes(volume_name=None, target_size=(256,64)):
    _min, _max  = None, None
    if target_size==(256,64):
        if "Left-Lung" in volume_name:
            _min, _max = 303, 5931
        elif "Right-Lung" in volume_name:
            _min, _max  = 302, 5948
        elif "Heart" in volume_name:
            _min, _max  = 300, 3756
        else: 
            print(f"wrong volume name: {volume_name}")
    elif target_size==(224,224):
        if "Left-Lung" in volume_name:
            _min, _max = 934, 18153
        elif "Right-Lung" in volume_name:
            _min, _max  = 912, 18218
        elif "Heart" in volume_name:
            _min, _max  = 897, 11522
        else: 
            print(f"wrong volume name: {volume_name}")
    return  _min, _max 

def get_norm_volumes(
        seg,
        volume_name,
):
    _min, _max = get_min_max_valumes(
        volume_name = volume_name,
        target_size=seg.size()[-2:],
    )

    sum_seg = torch.sum(seg, dim=(1,2,3), keepdim=False) # dim set for (B, C, H, W) segmentation

    norm_seg = (sum_seg-_min) / _max
    return norm_seg


In [57]:
get_norm_volumes(seg=seg, volume_name="Right-Lung_volume")

tensor([1.3750, 1.3928, 1.3814, 1.3924, 1.3709, 1.3850, 1.3418, 1.3728, 1.3676,
        1.3613, 1.3561, 1.4082, 1.3965, 1.3671, 1.3851, 1.3785],
       grad_fn=<DivBackward0>)